# Task Adherence Evaluator

### Getting Started

This sample demonstrates how to use Intent Resolution Evaluator
Before running the sample:
```bash
pip install azure-ai-projects azure-identity azure-ai-evaluation
```
Set these environment variables with your own values:
1) **PROJECT_CONNECTION_STRING** - The project connection string, as found in the overview page of your Azure AI Foundry project.
2) **MODEL_DEPLOYMENT_NAME** - The deployment name of the AI model, as found under the "Name" column in the "Models + endpoints" tab in your Azure AI Foundry project.
3) **AZURE_OPENAI_ENDPOINT** - Azure Open AI Endpoint to be used for evaluation.
4) **AZURE_OPENAI_API_KEY** - Azure Open AI Key to be used for evaluation.
5) **AZURE_OPENAI_API_VERSION** - Azure Open AI Api version to be used for evaluation.
6) **AZURE_SUBSCRIPTION_ID** - Azure Subscription Id of Azure AI Project
7) **PROJECT_NAME** - Azure AI Project Name
8) **RESOURCE_GROUP_NAME** - Azure AI Project Resource Group Name

The Task Adherence evaluator measures how well the agent adheres to their assigned tasks or predefined goal.

The scoring is on a 1-5 integer scale and is as follows:

  - Score 1: Fully Inadherent
  - Score 2: Barely Adherent
  - Score 3: Moderately Adherent
  - Score 4: Mostly Adherent
  - Score 5: Fully Adherent

The evaluation requires the following inputs:

  - Query    : The user query. Either a string with a user request or a list of messages with previous requests from the user and responses from the assistant, potentially including a system message.
  - Response : The response to be evaluated. Either a string or a message with the response from the agent to the last user query.

There is a third optional parameter:
  - ToolDefinitions : The list of tool definitions the agent can call. This may be useful for the evaluator to better assess if the right tool was called to adhere to user intent.

### Initialize Task Adherence Evaluator


In [ ]:
import os
from azure.identity import DefaultAzureCredential
from azure.ai.evaluation import TaskAdherenceEvaluator, AzureOpenAIModelConfiguration
from pprint import pprint

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["MODEL_DEPLOYMENT_NAME"],
)
task_adherence_evaluator = TaskAdherenceEvaluator(model_config)

### Samples

#### Evaluating query and response as string

In [ ]:
#Failure example, there's only a vague adherence to the task
result = task_adherence_evaluator(query="What are the best practices for maintaining a healthy rose garden during the summer?",
                                  response="Make sure to water your roses regularly and trim them occasionally.",
                                 )
pprint(result)

In [ ]:
#Success example, full adherence to the task
result = task_adherence_evaluator(query="What are the best practices for maintaining a healthy rose garden during the summer?",
                                  response="For optimal summer care of your rose garden, start by watering deeply early in the morning to ensure the roots are well-hydrated without encouraging fungal growth. Apply a 2-3 inch layer of organic mulch around the base of the plants to conserve moisture and regulate soil temperature. Fertilize with a balanced rose fertilizer every 4–6 weeks to support healthy growth. Prune away any dead or diseased wood to promote good air circulation, and inspect regularly for pests such as aphids or spider mites, treating them promptly with an appropriate organic insecticidal soap. Finally, ensure that your roses receive at least 6 hours of direct sunlight daily for robust flowering.",
                                 )
pprint(result)

#### Evaluating query and response as list of messages

In [ ]:
query = [
            {
                "role": "system",
                "content": "You are an expert in literature and at provid can provide book recommendations."
            },
            {
                "createdAt": "2025-03-14T08:00:00Z",
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "I love historical fiction. Can you recommend a good book from that genre?"
                    }
                ]
            }
        ]

response =  [
                {
                    "createdAt": "2025-03-14T08:00:05Z",
                    "role": "assistant",
                    "content": [
                        {
                            "type": "text",
                            "text": "Let me fetch a recommendation for historical fiction."
                        }
                    ]
                },
                {
                    "createdAt": "2025-03-14T08:00:10Z",
                    "role": "assistant",
                    "content": [
                        {
                            "type": "tool_call",
                            "tool_call_id": "tool_call_20250314_001",
                            "name": "get_book",
                            "arguments": {
                                "genre": "historical fiction"
                            }
                        }
                    ]
                },
                {
                    "createdAt": "2025-03-14T08:00:15Z",
                    "role": "tool",
                    "tool_call_id": "tool_call_20250314_001",
                    "content": [
                        {
                            "type": "tool_result",
                            "tool_result": "{ \"book\": { \"title\": \"The Pillars of the Earth\", \"author\": \"Ken Follett\", \"summary\": \"A captivating tale set in medieval England that weaves historical events with personal drama.\" } }"
                        }
                    ]
                },
                {
                    "createdAt": "2025-03-14T08:00:20Z",
                    "role": "assistant",
                    "content": [
                        {
                            "type": "text",
                            "text": "Based on our records, I recommend 'The Pillars of the Earth' by Ken Follett. This novel is an excellent example of historical fiction with a rich narrative and well-developed characters. Would you like more details or another suggestion?"
                        }
                    ]
                }
       ]

tool_definitions = [
                    {
                        "name": "get_book",
                        "description": "Retrieve a book recommendation for a specified genre.",
                        "parameters": {
                            "type": "object",
                            "properties": {
                                "genre": {
                                    "type": "string",
                                    "description": "The genre for which a book recommendation is requested."
                                }
                            }
                        }
                    }
                ]

result = task_adherence_evaluator(query=query,
                                  response=response,
                                  tool_definitions=tool_definitions,
                                 )
pprint(result)